# CellChat

In [ ]:
library_load <- suppressMessages(
    
    list(
        
        # CellTalk
        library(CellChat), 
        
        # Seurat 
        library(Seurat), 
        
        # Data 
        library(tidyverse), 
        
        # Plotting 
        library(ComplexHeatmap), 
        library(circlize), 
        library(viridis), 
        library(ggplotify), 
        library(ggrepel)
        
    )
)

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
options(warn=-1, stringsAsFactors=FALSE)

In [ ]:
ht_opt$message=FALSE # ComplexHeatmap 

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")
source("bin/cell_type.R")

In [ ]:
# Plotting Theme
ggplot2::theme_set(theme_global_set()) # From project global source()

# Parameter and files 

In [ ]:
so_file <- "data/object/pp.rds"

In [ ]:
compute_cellchat <- TRUE

# Import 

In [ ]:
so <- readRDS(so_file)

In [ ]:
so$cell_type_main <- factor(so$cell_type_main, levels=cell_type_main_order)
so$cell_type_fine <- factor(so$cell_type_fine, levels=cell_type_fine_order)

## Pre-process cellchat objects for each treatment condition 

In [ ]:
cellchat_pp <- function(so, group_by) {
    
    cnt <- as.matrix(GetAssayData(so, assay="RNA", slot="data"))
    cnt <- cnt[rowSums(cnt)>0, ]
    
    cellchat <- createCellChat(object=cnt, meta=so@meta.data, group.by=group_by)
    cellchat <- setIdent(cellchat, ident.use=group_by)
    cellchat@DB <- CellChatDB.mouse
    cellchat <- subsetData(cellchat)
    cellchat <- identifyOverExpressedGenes(cellchat, thresh.p=1)
    cellchat <- identifyOverExpressedInteractions(cellchat)
    cellchat <- computeCommunProb(cellchat, type="truncatedMean", trim=0.01)
    cellchat <- computeCommunProbPathway(cellchat)
    cellchat <- aggregateNet(cellchat)
    
    return(cellchat)
}

In [ ]:
if(compute_cellchat) {
    
    cellchat_main_nacl <- cellchat_pp(subset(so, subset=treatment=="NaCl"), group_by="cell_type_main")
    saveRDS(cellchat_main_nacl, "data/object/cellchat/cellchat_main_nacl.rds")

} else {
    
    cellchat_main_nacl <- readRDS("data/object/cellchat/cellchat_main_nacl.rds")
    
}

In [ ]:
if(compute_cellchat) {
    
    cellchat_main_cpg <- cellchat_pp(subset(so, subset=treatment=="CpG"), group_by="cell_type_main")
    saveRDS(cellchat_main_cpg, "data/object/cellchat/cellchat_main_cpg.rds")

} else {
    
    cellchat_main_cpg <- readRDS("data/object/cellchat/cellchat_main_cpg.rds")
    
}

In [ ]:
if(compute_cellchat) {
    
    cellchat_fine_nacl <- cellchat_pp(subset(so, subset=treatment=="NaCl"), group_by="cell_type_fine")
    saveRDS(cellchat_fine_nacl, "data/object/cellchat/cellchat_fine_nacl.rds")

} else {
    
    cellchat_fine_nacl <- readRDS("data/object/cellchat/cellchat_fine_nacl.rds")
    
}

In [ ]:
if(compute_cellchat) {
    
    cellchat_fine_cpg <- cellchat_pp(subset(so, subset=treatment=="CpG"), group_by="cell_type_fine")
    saveRDS(cellchat_fine_cpg, "data/object/cellchat/cellchat_fine_cpg.rds")

} else {
    
    cellchat_fine_cpg <- readRDS("data/object/cellchat/cellchat_fine_cpg.rds")
    
}

# Merge CellChat data objects 

In [ ]:
cellchat_main_list <- list(nacl=cellchat_main_nacl, cpg=cellchat_main_cpg)
cellchat_main <- mergeCellChat(cellchat_main_list, add.names=names(cellchat_main_list))

In [ ]:
cellchat_fine_list <- list(nacl=cellchat_fine_nacl, cpg=cellchat_fine_cpg)
cellchat_fine <- mergeCellChat(cellchat_fine_list, add.names=names(cellchat_fine_list))

In [ ]:
cellchat_fine_cpg@LR[[1]][cellchat_fine_cpg@LR[[1]]$ligand=="Cxcl2", ]

# NetVisual circle plot

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)

groupSize <- as.numeric(table(cellchat_main_cpg@idents))
par(mfrow=c(1,2), xpd=TRUE)
catch <- netVisual_circle(cellchat_main_cpg@net$count, vertex.weight=groupSize, weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_main_short), title.name="Number of interactions (CpG)")
catch <- netVisual_circle(cellchat_main_cpg@net$weight, vertex.weight=groupSize, weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_main_short), title.name="Interaction weights/strength (CpG)")

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)

par(mfrow=c(1,2), xpd=TRUE)
catch <- netVisual_diffInteraction(cellchat_main, measure="count", weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_main_short), color.edge=rev(unlist(color$treatment)), title.name="Number of interactions (Difference)")
catch <- netVisual_diffInteraction(cellchat_main, measure="weight", weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_main_short), color.edge=rev(unlist(color$treatment)), title.name="Interaction weights/strength (Difference)")

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)

groupSize <- as.numeric(table(cellchat_fine_nacl@idents))
par(mfrow=c(1,2), xpd=TRUE)
catch <- netVisual_circle(cellchat_fine_nacl@net$count, vertex.weight=groupSize, weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_fine_short), title.name="Number of interactions (NaCl)")
catch <- netVisual_circle(cellchat_fine_nacl@net$weight, vertex.weight=groupSize, weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_fine_short), title.name="Interaction weights/strength (NaCl)")

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)

groupSize <- as.numeric(table(cellchat_fine_cpg@idents))
par(mfrow=c(1,2), xpd=TRUE)
catch <- netVisual_circle(cellchat_fine_cpg@net$count, vertex.weight=groupSize, weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_fine_short), title.name="Number of interactions (CpG)")
catch <- netVisual_circle(cellchat_fine_cpg@net$weight, vertex.weight=groupSize, weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_fine_short), title.name="Interaction weights/strength (CpG)")

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)

par(mfrow=c(1,2), xpd=TRUE)
catch <- netVisual_diffInteraction(cellchat_fine, measure="count", weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_fine_short), color.edge=rev(unlist(color$treatment)), title.name="Number of interactions (Difference)")
catch <- netVisual_diffInteraction(cellchat_fine, measure="weight", weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_fine_short), color.edge=rev(unlist(color$treatment)), title.name="Interaction weights/strength (Difference)")

# NetVisual heatmap 

In [ ]:
options(repr.plot.width=15, repr.plot.height=5)

p_1 <- netVisual_heatmap(cellchat_main_nacl, measure="count", color.use=unlist(color$cell_type_main_short), color.heatmap=c("white", "black"), title.name="Number of interaction NaCl") %>% as.ggplot() + theme(aspect.ratio=1)
p_2 <- netVisual_heatmap(cellchat_main_cpg, measure="count", color.use=unlist(color$cell_type_main_short), color.heatmap=c("white", "black"), title.name="Number of interaction CpG") %>% as.ggplot() + theme(aspect.ratio=1)
p_3 <- netVisual_heatmap(cellchat_main, measure="count", color.use=unlist(color$cell_type_main_short), color.heatmap=unlist(color$treatment), title.name="Number of interaction Diff") %>% as.ggplot() + theme(aspect.ratio=1)

p_1 + p_2 + p_3

In [ ]:
options(repr.plot.width=15, repr.plot.height=5)

p_1 <- netVisual_heatmap(cellchat_fine_nacl, measure="count", color.use=unlist(color$cell_type_fine_short), color.heatmap=c("white", "black"), title.name="Number of interaction NaCl") %>% as.ggplot() + theme(aspect.ratio=1)
p_2 <- netVisual_heatmap(cellchat_fine_cpg, measure="count", color.use=unlist(color$cell_type_fine_short), color.heatmap=c("white", "black"), title.name="Number of interaction CpG") %>% as.ggplot() + theme(aspect.ratio=1)
p_3 <- netVisual_heatmap(cellchat_fine, measure="count", color.use=unlist(color$cell_type_fine_short), color.heatmap=unlist(color$treatment), title.name="Number of interaction Diff") %>% as.ggplot() + theme(aspect.ratio=1)

p_1 + p_2 + p_3

# Aggregated cell-cell communication network

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)

groupSize <- as.numeric(table(cellchat_main_nacl@idents))
mat <- cellchat_main_nacl@net$count
par(mfrow=c(3, 5), xpd=TRUE)
for (i in 1:nrow(mat)) {
    
    mat2 <- matrix(0, nrow=nrow(mat), ncol=ncol(mat), dimnames=dimnames(mat))
    mat2[i, ] <- mat[i, ] 
    netVisual_circle(mat2, vertex.weight=groupSize, weight.scale=TRUE, edge.weight.max=max(mat), title.name=rownames(mat)[i], arrow.size=0, arrow.width=0, color.use=unlist(color$cell_type_main_short)) 
}

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)

groupSize <- as.numeric(table(cellchat_main_cpg@idents))
mat <- cellchat_main_cpg@net$count
par(mfrow=c(3, 5), xpd=TRUE)
for (i in 1:nrow(mat)) {
    
    mat2 <- matrix(0, nrow=nrow(mat), ncol=ncol(mat), dimnames=dimnames(mat))
    mat2[i, ] <- mat[i, ]
    netVisual_circle(mat2, vertex.weight=groupSize, weight.scale=TRUE, edge.weight.max=max(mat), title.name=rownames(mat)[i], arrow.size=0, arrow.width=0, color.use=unlist(color$cell_type_main_short))
}

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)

par(mfrow=c(3, 5), xpd=TRUE)
for (i in levels(cellchat_main_nacl@idents)) {
    
    netVisual_diffInteraction(cellchat_main, measure="count", weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_main_short), color.edge=rev(unlist(color$treatment)), sources.use=i, title.name=c(i))
    
}

In [ ]:
options(repr.plot.width=25, repr.plot.height=25)

groupSize <- as.numeric(table(cellchat_fine_nacl@idents))
mat <- cellchat_fine_nacl@net$count
par(mfrow=c(5, 5), xpd=TRUE)
for (i in 1:nrow(mat)) {
    
    mat2 <- matrix(0, nrow=nrow(mat), ncol=ncol(mat), dimnames=dimnames(mat))
    mat2[i, ] <- mat[i, ]
    netVisual_circle(mat2, vertex.weight=groupSize, weight.scale=TRUE, edge.weight.max=max(mat), title.name=rownames(mat)[i], arrow.size=0, arrow.width=0, color.use=unlist(color$cell_type_fine_short))
    
}

In [ ]:
options(repr.plot.width=25, repr.plot.height=25)

groupSize <- as.numeric(table(cellchat_fine_cpg@idents))
mat <- cellchat_fine_cpg@net$count
par(mfrow=c(5, 5), xpd=TRUE)
for (i in 1:nrow(mat)) {
    
    mat2 <- matrix(0, nrow=nrow(mat), ncol=ncol(mat), dimnames=dimnames(mat))
    mat2[i, ] <- mat[i, ]
    netVisual_circle(mat2, vertex.weight=groupSize, weight.scale=TRUE, edge.weight.max=max(mat), title.name=rownames(mat)[i], arrow.size=0, arrow.width=0, color.use=unlist(color$cell_type_fine_short))
    
}

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)

par(mfrow=c(3, 5), xpd=TRUE)
for (i in levels(cellchat_fine_nacl@idents)) {
    
    netVisual_diffInteraction(cellchat_fine, measure="count", weight.scale=TRUE, label.edge=FALSE, color.use=unlist(color$cell_type_fine_short), color.edge=rev(unlist(color$treatment)), sources.use=i, title.name=c(i))
    
}

# Visualize cell-cell communication mediated by multiple ligand-receptors

In [ ]:
cluster_main_ery <- c("MEP", "Pro-EB", "EB")
cluster_fine_ery <- c("MEP (1)", "MEP (2)", "MEP (3)", "Pro-EB (1)", "Pro-EB (2)", "Pro-EB (3)", "Pro-EB (4)", "EB (1)", "EB (2)", "EB (3)","EB (4)", "EB (5)")

cluster_main_m <- c("Mo (1)", "Mo (2)", "Mo (3)", "Mo (4)", "cDC2 (1)", "cDC1", "cDC2 (2)", "RPM", "MoRPM")
cluster_fine_m <- c("Mo (1)", "Mo (2)", "Mo (3)", "Mo (4)", "cDC2 (1)", "cDC1", "cDC2 (2)", "RPM", "MoRPM")

### Main NaCl

In [ ]:
options(repr.plot.width=15, repr.plot.height=3*7.5)

nvb_1 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_m[1], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[1], "ligand (NaCl)"))
nvb_2 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_m[2], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[2], "ligand (NaCl)"))
nvb_3 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_m[3], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[3], "ligand (NaCl)"))
nvb_4 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_m[4], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[4], "ligand (NaCl)"))
nvb_5 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_m[5], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[5], "ligand (NaCl)"))
nvb_6 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_m[6], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[6], "ligand (NaCl)"))
nvb_7 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_m[7], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[7], "ligand (NaCl)"))
nvb_8 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_m[8], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[8], "ligand (NaCl)"))
nvb_9 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_m[9], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[9], "ligand (NaCl)"))

nvb_1 + nvb_2 + nvb_3 + nvb_4 + nvb_5 + nvb_6 + nvb_7 + nvb_8 + nvb_9

In [ ]:
options(repr.plot.width=15, repr.plot.height=2*7.5)

nvb_1 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_ery[1], targets.use=cluster_main_m) + ggtitle(paste(cluster_main_ery[1], "ligand (NaCl)"))
nvb_2 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_ery[2], targets.use=cluster_main_m) + ggtitle(paste(cluster_main_ery[2], "ligand (NaCl)"))
nvb_3 <- netVisual_bubble(cellchat_main_nacl, thresh=0.01, sources.use=cluster_main_ery[3], targets.use=cluster_main_m) + ggtitle(paste(cluster_main_ery[3], "ligand (NaCl)"))

nvb_1 + nvb_2 + nvb_3

### Main CpG

In [ ]:
options(repr.plot.width=15, repr.plot.height=3*7.5)

nvb_1 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_m[1], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[1], "ligand (NaCl)"))
nvb_2 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_m[2], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[2], "ligand (NaCl)"))
nvb_3 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_m[3], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[3], "ligand (NaCl)"))
nvb_4 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_m[4], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[4], "ligand (NaCl)"))
nvb_5 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_m[5], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[5], "ligand (NaCl)"))
nvb_6 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_m[6], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[6], "ligand (NaCl)"))
nvb_7 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_m[7], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[7], "ligand (NaCl)"))
nvb_8 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_m[8], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[8], "ligand (NaCl)"))
nvb_9 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_m[9], targets.use=cluster_main_ery) + ggtitle(paste(cluster_main_m[9], "ligand (NaCl)"))

nvb_1 + nvb_2 + nvb_3 + nvb_4 + nvb_5 + nvb_6 + nvb_7 + nvb_8 + nvb_9

In [ ]:
options(repr.plot.width=15, repr.plot.height=2*7.5)

nvb_1 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_ery[1], targets.use=cluster_main_m) + ggtitle(paste(cluster_main_ery[1], "ligand (NaCl)"))
nvb_2 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_ery[2], targets.use=cluster_main_m) + ggtitle(paste(cluster_main_ery[2], "ligand (NaCl)"))
nvb_3 <- netVisual_bubble(cellchat_main_cpg, thresh=0.01, sources.use=cluster_main_ery[3], targets.use=cluster_main_m) + ggtitle(paste(cluster_main_ery[3], "ligand (NaCl)"))

nvb_1 + nvb_2 + nvb_3

### Fine NaCl

In [ ]:
options(repr.plot.width=15, repr.plot.height=3*7.5)

nvb_1 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_m[1], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[1], "ligand (NaCl)"))
nvb_2 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_m[2], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[2], "ligand (NaCl)"))
nvb_3 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_m[3], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[3], "ligand (NaCl)"))
nvb_4 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_m[4], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[4], "ligand (NaCl)"))
nvb_5 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_m[5], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[5], "ligand (NaCl)"))
nvb_6 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_m[6], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[6], "ligand (NaCl)"))
nvb_7 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_m[7], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[7], "ligand (NaCl)"))
nvb_8 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_m[8], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[8], "ligand (NaCl)"))
nvb_9 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_m[9], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[9], "ligand (NaCl)"))

nvb_1 + nvb_2 + nvb_3 + nvb_4 + nvb_5 + nvb_6 + nvb_7 + nvb_8 + nvb_9

In [ ]:
options(repr.plot.width=15, repr.plot.height=4*7.5)

nvb_1 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[1], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[1], "ligand (NaCl)"))
nvb_2 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[2], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[2], "ligand (NaCl)"))
nvb_3 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[3], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[3], "ligand (NaCl)"))
nvb_4 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[4], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[4], "ligand (NaCl)"))
nvb_5 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[5], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[5], "ligand (NaCl)"))
nvb_6 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[6], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[6], "ligand (NaCl)"))
nvb_7 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[7], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[7], "ligand (NaCl)"))
nvb_8 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[8], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[8], "ligand (NaCl)"))
nvb_9 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[9], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[9], "ligand (NaCl)"))
nvb_10 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[10], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[10], "ligand (NaCl)"))
nvb_11 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[11], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[11], "ligand (NaCl)"))
nvb_12 <- netVisual_bubble(cellchat_fine_nacl, thresh=0.01, sources.use=cluster_fine_ery[12], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[12], "ligand (NaCl)"))

nvb_1 + nvb_2 + nvb_3 + nvb_4 + nvb_5 + nvb_6 + nvb_7 + nvb_8 + nvb_9 + nvb_10 + nvb_11 + nvb_12 

### Fine CpG

In [ ]:
options(repr.plot.width=15, repr.plot.height=3*7.5)

nvb_1 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_m[1], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[1], "ligand (CpG)"))
nvb_2 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_m[2], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[2], "ligand (CpG)"))
nvb_3 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_m[3], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[3], "ligand (CpG)"))
nvb_4 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_m[4], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[4], "ligand (CpG)"))
nvb_5 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_m[5], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[5], "ligand (CpG)"))
nvb_6 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_m[6], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[6], "ligand (CpG)"))
nvb_7 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_m[7], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[7], "ligand (CpG)"))
nvb_8 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_m[8], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[8], "ligand (CpG)"))
nvb_9 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_m[9], targets.use=cluster_fine_ery) + ggtitle(paste(cluster_fine_m[9], "ligand (CpG)"))

nvb_1 + nvb_2 + nvb_3 + nvb_4 + nvb_5 + nvb_6 + nvb_7 + nvb_8 + nvb_9

In [ ]:
options(repr.plot.width=15, repr.plot.height=5*7.5)

nvb_1 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[1], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[1], "ligand (CpG)")) 
nvb_2 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[2], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[2], "ligand (CpG)"))
nvb_3 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[3], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[3], "ligand (CpG)"))
nvb_4 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[4], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[4], "ligand (CpG)"))
nvb_5 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[5], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[5], "ligand (CpG)"))
nvb_6 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[6], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[6], "ligand (CpG)"))
nvb_7 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[7], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[7], "ligand (CpG)"))
nvb_8 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[8], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[8], "ligand (CpG)"))
nvb_9 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[9], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[9], "ligand (CpG)"))
nvb_10 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[10], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[10], "ligand (CpG)"))
nvb_11 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[11], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[11], "ligand (CpG)"))
nvb_12 <- netVisual_bubble(cellchat_fine_cpg, thresh=0.01, sources.use=cluster_fine_ery[12], targets.use=cluster_fine_m) + ggtitle(paste(cluster_fine_ery[12], "ligand (CpG)"))

nvb_1 + nvb_2 + nvb_3 + nvb_4 + nvb_5 + nvb_6 + nvb_7 + nvb_8 + nvb_9 + nvb_10 + nvb_11 + nvb_12 

# Visualize cell-cell communication mediated by multiple ligand-receptors (Differential) 

### Main labels

In [ ]:
options(repr.plot.width=20, repr.plot.height=10*7.5)

nvb_1 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_m[1], targets.use=cluster_main_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_m[1], "ligand (Diff)"))
nvb_2 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_m[2], targets.use=cluster_main_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_m[2], "ligand (Diff)"))
nvb_3 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_m[3], targets.use=cluster_main_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_m[3], "ligand (Diff)"))
nvb_4 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_m[4], targets.use=cluster_main_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_m[4], "ligand (Diff)"))
nvb_5 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_m[5], targets.use=cluster_main_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_m[5], "ligand (Diff)"))
nvb_6 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_m[6], targets.use=cluster_main_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_m[6], "ligand (Diff)"))
nvb_7 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_m[7], targets.use=cluster_main_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_m[7], "ligand (Diff)"))
nvb_8 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_m[8], targets.use=cluster_main_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_m[8], "ligand (Diff)"))
nvb_9 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_m[9], targets.use=cluster_main_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_m[9], "ligand (Diff)"))

nvb_1 + nvb_2 + nvb_3 + nvb_4 + nvb_5 + nvb_6 + nvb_7 + nvb_8 + nvb_9

In [ ]:
options(repr.plot.width=15, repr.plot.height=10*7.5)

nvb_1 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_ery[1], targets.use=cluster_main_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_ery[1], "ligand (Diff)"))
nvb_2 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_ery[2], targets.use=cluster_main_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_ery[2], "ligand (Diff)"))
nvb_3 <- netVisual_bubble(cellchat_main, thresh=0.01, sources.use=cluster_main_ery[3], targets.use=cluster_main_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_main_ery[3], "ligand (Diff)"))

nvb_1 + nvb_2 + nvb_3

### Fine labels

In [ ]:
CellChatDB <- CellChat::CellChatDB.mouse
interaction <- CellChatDB$interaction

In [ ]:
interaction <- interaction[grep("Ackr1", interaction$receptor), "interaction_name", drop=FALSE]

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)

nvb_1 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_m[1], pairLR.use=interaction, targets.use=cluster_fine_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_m[1], "ligand (Diff)")) + theme_global_set() + theme(legend.position="none", axis.text=element_text(size=8), axis.text.x=element_blank()) + xlab("") + ylab("")
nvb_2 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_m[2], pairLR.use=interaction, targets.use=cluster_fine_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_m[2], "ligand (Diff)")) + theme_global_set() + theme(legend.position="none", axis.text=element_text(size=8), axis.text.x=element_blank()) + xlab("") + ylab("")
nvb_3 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_m[3], pairLR.use=interaction, targets.use=cluster_fine_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_m[3], "ligand (Diff)")) + theme_global_set() + theme(legend.position="none", axis.text=element_text(size=8), axis.text.x=element_blank()) + xlab("") + ylab("")
nvb_4 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_m[4], pairLR.use=interaction, targets.use=cluster_fine_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_m[4], "ligand (Diff)")) + theme_global_set() + theme(legend.position="none", axis.text=element_text(size=8), axis.text.x=element_blank()) + xlab("") + ylab("")
nvb_5 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_m[5], pairLR.use=interaction, targets.use=cluster_fine_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_m[5], "ligand (Diff)")) + theme_global_set() + theme(legend.position="none", axis.text=element_text(size=8), axis.text.x=element_blank()) + xlab("") + ylab("")
nvb_6 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_m[6], pairLR.use=interaction, targets.use=cluster_fine_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_m[6], "ligand (Diff)")) + theme_global_set() + theme(legend.position="none", axis.text=element_text(size=8), axis.text.x=element_blank()) + xlab("") + ylab("")
nvb_7 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_m[7], pairLR.use=interaction, targets.use=cluster_fine_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_m[7], "ligand (Diff)")) + theme_global_set() + theme(legend.position="none", axis.text=element_text(size=8)) + xlab("") + ylab("")
nvb_8 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_m[8], pairLR.use=interaction, targets.use=cluster_fine_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_m[8], "ligand (Diff)")) + theme_global_set() + theme(legend.position="none", axis.text=element_text(size=8)) + xlab("") + ylab("")
nvb_9 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_m[9], pairLR.use=interaction, targets.use=cluster_fine_ery, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_m[9], "ligand (Diff)")) + theme_global_set() + theme(legend.position="none", axis.text=element_text(size=8)) + xlab("") + ylab("")

nvb_all <- nvb_1 + nvb_2 + nvb_3 + nvb_4 + nvb_5 + nvb_6 + nvb_7 + nvb_8 + nvb_9
nvb_all

In [ ]:
pdf("result/figures/cellchat_netvisual_m_to_ery_ackr1.pdf", widt=10, height=10)
nvb_all
dev.off()

png("result/figures/cellchat_netvisual_m_to_ery_ackr1.png", width=1200, height=500, units="px")
nvb_all
dev.off()

In [ ]:
options(repr.plot.width=20, repr.plot.height=7.5)

nvb_1 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[1], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[1], "ligand (Diff)")) 
nvb_2 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[2], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[2], "ligand (Diff)"))
nvb_3 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[3], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[3], "ligand (Diff)"))
nvb_4 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[4], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[4], "ligand (Diff)"))
nvb_5 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[5], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[5], "ligand (Diff)"))
nvb_6 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[6], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[6], "ligand (Diff)"))
nvb_7 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[7], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[7], "ligand (Diff)"))
nvb_8 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[8], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[8], "ligand (Diff)"))
nvb_9 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[9], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[9], "ligand (Diff)"))
nvb_10 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[10], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[10], "ligand (Diff)"))
nvb_11 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[11], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[11], "ligand (Diff)"))
nvb_12 <- netVisual_bubble(cellchat_fine, thresh=0.01, sources.use=cluster_fine_ery[12], pairLR.use=interaction, targets.use=cluster_fine_m, comparison=c(1, 2), max.dataset=NULL, color.text.use=FALSE) + ggtitle(paste(cluster_fine_ery[12], "ligand (Diff)"))

nvb_1 + nvb_2 + nvb_3 + nvb_4 + nvb_5 + nvb_6 + nvb_7 + nvb_8 + nvb_9 + nvb_10 + nvb_11 + nvb_12 

# Compare the major sources and targets in 2D space

In [ ]:
cellchat_fine_nacl <- netAnalysis_computeCentrality(cellchat_fine_nacl)
cellchat_fine_cpg <- netAnalysis_computeCentrality(cellchat_fine_cpg)
cellchat_fine_list <- list(nacl=cellchat_fine_nacl, cpg=cellchat_fine_cpg)

In [ ]:
options(repr.plot.width=15, repr.plot.height=7.5)

num_link <- sapply(cellchat_fine_list, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})
weight_limits <- c(min(num_link), max(num_link)) # control the dot size in the different datasets

title <- c("NaCl signaling", "CpG signaling")

dplot <- list()
for (i in 1:length(cellchat_fine_list)) {
    
    dplot_i <- netAnalysis_signalingRole_scatter(cellchat_fine_list[[i]], weight.MinMax=weight_limits, do.label=FALSE, show.legend=TRUE, dot.alpha=0.2) + 
        geom_label_repel(mapping=aes(label=labels, colour=labels), segment.color="black", force=50, force_pull=1, max.overlaps=getOption("ggrepel.max.overlaps", default=100), label.size=NA, fill=alpha(c("white"), 0), size=4) + 
        ggtitle(title[i]) + 
        scale_color_manual(values=color$cell_type_fine_short) + theme_global_set()
    
    dplot_i <- remove_geom(dplot_i, "GeomTextRepel")
    
    dplot_i <- dplot_i + theme_global_set()
    
    dplot[[i]] <- dplot_i
}

patchwork::wrap_plots(plots=dplot)

In [ ]:
pdf("result/figures/cellchat_dp_signaling_nacl.pdf", widt=10, height=10)
dplot[[1]]
dev.off()

png("result/figures/cellchat_dp_signaling_nacl.png", width=600, height=400, units="px")
dplot[[1]]
dev.off()

In [ ]:
pdf("result/figures/cellchat_dp_signaling_cpg.pdf", widt=10, height=10)
dplot[[2]]
dev.off()

png("result/figures/cellchat_dp_signaling_cpg.png", width=600, height=400, units="px")
dplot[[2]]
dev.off()